In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
!cd '/content/gdrive/My Drive/Object_Detection_Transformers/main-repo/dataset/area_of_stages'

In [10]:
import numpy as np
import os

In [16]:
def bbox_iou(bbox_a, bbox_b):
    """Calculate the Intersection of Unions (IoUs) between bounding boxes.

    IoU is calculated as a ratio of area of the intersection
    and area of the union.

    both inputs and output should be python list
    """
    ##########--WRITE YOUR CODE HERE--##########

    a = np.array(bbox_a)
    b = np.array(bbox_b)
    a = a.repeat(b.shape[0], axis = 0)

    top_left_a = a[ : , [0,1]] 
    top_left_b = b[ : , [0,1]]
    bottom_right_a = a[ : , [2,3]] 
    bottom_right_b = b[ : , [2,3]]
    max_pts = np.maximum(top_left_a , top_left_b)
    min_pts = np.minimum(bottom_right_a , bottom_right_b)

    inter_pts = np.concatenate((max_pts , min_pts) , axis = 1)

    inter_dim = np.stack( [inter_pts[: , 2] -  inter_pts[: , 0] , inter_pts[: , 3] -  inter_pts[: , 1] ] , axis = 0).T
    inter_dim[inter_dim < 0] = 0 
    inter = np.multiply( inter_dim[:, 0] , inter_dim[:,1])

    gt_area = (bbox_a[0][2] - bbox_a[0][0]) * (bbox_a[0][3] - bbox_a[0][1])

    bbox_b_dim = np.stack( [b[: , 2] -  b[: , 0] , b[: , 3] -  b[: , 1] ] , axis = 0).T
    bbox_b_areas = np.multiply( bbox_b_dim[:, 0] , bbox_b_dim[:,1])

    union = bbox_b_areas - inter + gt_area

    iou = [( inter / union ).tolist()]



    ##########-------END OF CODE-------########## 
    return iou


In [70]:
def area(box):
  return (box[3] - box[1]) * (box[2] - box[0])


In [78]:
def get_unique_boxes(path_annot):

    annot_paths = [
      f for f in os.listdir(path_annot) if os.path.isfile(os.path.join(path_annot, f))
    ]
    image_size = 224  

    images, targets = [], []

    # loop over the annotations and images, preprocess them and store in lists

    elkhorn = []
    staghorn = []

    for i in range(len(annot_paths)):
      f = open(path_annot +  annot_paths[i] , "r")
      boxes = []
      line = f.readline()
      while line:
        coordinates = line.split()
        coral_type = int(float(coordinates[0]))
        x_center, y_center ,box_width, box_height  = float(coordinates[1]) , float(coordinates[2]) , float(coordinates[3]) , float(coordinates[4] )
        top_left_x = x_center - box_width / 2 
        top_left_y = y_center - box_height / 2
        bottom_right_x = x_center + box_width / 2 
        bottom_right_y = y_center + box_height / 2

        boxes.append([top_left_x * image_size, top_left_y * image_size , bottom_right_x * image_size , bottom_right_y * image_size])
        line = f.readline()

      if(boxes == []):
        continue
      boxes_unique = [boxes.pop()]

      for box in boxes:
        iou = bbox_iou([box] , boxes_unique)[0]
        drop = False
        for val in iou:
          if(val > 0.8):
            drop = True
            break
        if(not drop):
          boxes_unique.append(box)
      if(coral_type == 0):
        elkhorn.extend(boxes_unique)
      else :
        staghorn.extend(boxes_unique)


      f.close()
    
    return elkhorn , staghorn




# Growth percentage of Corals

In [85]:
stage1_elk , stage1_stag = get_unique_boxes('/content/gdrive/My Drive/Object_Detection_Transformers/main-repo/dataset/area_of_stages/stage1/labels/')
stage2_elk , stage2_stag = get_unique_boxes('/content/gdrive/My Drive/Object_Detection_Transformers/main-repo/dataset/area_of_stages/stage2/labels/')

stage1_elk_area = sum([area(box) for box in stage1_elk])
stage2_elk_area = sum([area(box) for box in stage2_elk])

stage1_stag_area = sum([area(box) for box in stage1_stag])
stage2_stag_area = sum([area(box) for box in stage2_stag])


print("STAGE 1 : Area of Elkhorn " , round(stage1_elk_area , 2))
print("STAGE 2 : Area of Elkhorn " , round(stage2_elk_area , 2))

print("\n-------------------------------------------------------------------------\n")


print("STAGE 1 : Area of Staghorn " , round(stage1_stag_area , 2))
print("STAGE 2 : Area of Staghorn " , round(stage2_stag_area , 2))


print("\n-------------------------------------------------------------------------\n")

print("Growth percentage of Elkhorn" , round(((stage2_elk_area - stage1_elk_area) / stage1_elk_area) , 2) , "%")
print("Growth percentage of Staghorn" , round(((stage2_stag_area - stage1_stag_area) / stage1_stag_area) , 2) , "%")


STAGE 1 : Area of Elkhorn  11239.71
STAGE 2 : Area of Elkhorn  100743.88

-------------------------------------------------------------------------

STAGE 1 : Area of Staghorn  34726.3
STAGE 2 : Area of Staghorn  132252.97

-------------------------------------------------------------------------

Growth percentage of Elkhorn 7.96 %
Growth percentage of Staghorn 2.81 %
